# Run processing

Augment your data with processing algorithms.

## Step 1. Set up the notebook

### 1. Install dependencies

In [ ]:
!pip install up42-py --upgrade -q

import up42, pathlib
from up42 import processing_templates

### 2. Authenticate

Create a `credentials.json` file in a directory named `.up42` under your home directory by running the cell below. The path to the file will be `~/.up42/credentials.json`.

In [ ]:
# Define the credentials file path if it doesn't exist
credentials_file_path = pathlib.Path.home().joinpath(".up42/credentials.json")
credentials_file_path.parent.mkdir(parents=True, exist_ok=True)
credentials_file_path.touch(exist_ok=True)

# Print the path to the file
print(f"Credentials file is located at: {credentials_file_path}")

1. Click the link above to the created file and paste the following code:
    ```json
    {
        "username": "<your-email-address>",
        "password": "<your-password>"
    }
    ```
2. Retrieve the email address and password used for logging into the console. Use them as values for username and password.
3. Save the `credentials.json` file.
4. Check that the authentication was successful as follows:

In [ ]:
up42.authenticate(cfg_file=credentials_file_path)

## Step 2. Select a process

[Processes](https://docs.up42.com/processing/processes) are algorithms that you can apply to STAC items in your storage. There are two types of processes:

- Enhancement processes that pre-process imagery to improve its visual quality.
- Analytics processes that extract insights from imagery.

[Retrieve the class name](https://docs.up42.com/sdk/reference/sdk-processing-templates) of the process you want to run.

If you want to run the chosen process for the first time, review and accept its end-user license agreement (EULA) in one of the following ways:

- [Via the console](https://docs.up42.com/getting-started/account/eulas)
- [Via the API](https://docs.up42.com/developers/api-eulas)

If a EULA is updated, you need to re-accept it before creating your next job.

## Step 3. Prepare input parameters

### 1. Select input STAC items

Select input STAC items and add a title to your job template.

In [ ]:
# Add a job title, required for all processes
title = "Processing data over Berlin"

# For processes with one input item, select a single item
stac_item_id = "a32d4e56-2dd2-4094-981f-00d73edb7c45"

# Get a single item from the STAC client
stac_client = up42.stac_client()
stac_item = next(stac_client.get_items(stac_item_id))

# For processes with several input items, select multiple items
# stac_item1_id = "68567134-27ad-7bd7-4b65-d61adb11fc78"
# stac_item2_id = "c3de9ed8-f6e5-4bb5-a157-f6430ba756da"

# Get multiple items from the STAC client
# stac_client = up42.stac_client()
# stac_item1 = next(stac_client.get_items(stac_item1_id))
# stac_item2 = next(stac_client.get_items(stac_item2_id))

### 2. Create a job template

Create a template by specifying the values of the input parameters for the process you have selected.

In [ ]:
# Processes have different input parameters, but most of them only require a job title and a STAC item as DetectionAircraftOI
job_template = processing_templates.DetectionAircraftOI(
    title=title,
    item=stac_item
)

# This example required 2 input STAC items
# job_template = processing_templates.DetectionChangePleiadesHyperverge(
#    title=title,
#    items=[stac_item1, stac_item2]
# )

### 3. Validate your input

Check the validity of your job template.

In [ ]:
# Assert that the job is valid, print errors if not
if not job_template.is_valid:
  for error in job_template.errors:
    print(f"{error}\n")

## Step 4. Determine the cost

Check the cost of a valid job template.

In [ ]:
# Verify that the template cost is less than or equal to the price you selected
# This example checks the job estimation is below 100 credits
acceptable_cost = 100

if job_template.cost <= acceptable_cost:
  print(f"Template cost:       {job_template.cost.credits} credits")
  print(f"Template strategy:   {job_template.cost.strategy}")
  print(f"STAC item size:      {job_template.cost.size}")
  print(f"Unit of measurement: {job_template.cost.unit}")

else:
  print(f"Template cost is higher than {acceptable_cost} credits")


## Step 5. Run a process

In [ ]:
job = job_template.execute()

print(f"Job details for ID: {job.id}")
print(f"Process ID:         {job.process_id}")
print(f"Status:             {job.status.value}")
print(f"Price:              {job.credits or 'N/A'} credits")
print(f"Created at:         {job.created}")

## Step 6. Monitor a job

### Track job statuses

The `track` instance method allows you to track the job status until it changes to `CAPTURED` or `RELEASED`.

In [ ]:
# This will print a status log to your console every 20 seconds
# and will PAUSE the script here until the job is done
job.track(wait=20)

# This line will only run AFTER the job has finished tracking
print("\nTracking complete!")
print(f"The final job status is: {job.status.value}")

## Step 7. Download processing results

In a successfully completed job, get the resulting STAC collection.

In [ ]:
if job.status == up42.JobStatus.CAPTURED:
    print(f"STAC collection ID: {job.collection.id}")